<a href="https://www.kaggle.com/code/khaydaraliev97/binary-classification-with-a-bank-churn-dataset?scriptVersionId=185375258" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict

from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, roc_curve, auc, RocCurveDisplay

from scipy.stats import randint
import time
import warnings
warnings.simplefilter("ignore", category=PendingDeprecationWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)

In [37]:
url = '/kaggle/input/playground-series-s4e1/train.csv'
df = pd.read_csv(url)
df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165034 non-null  int64  
 1   CustomerId       165034 non-null  int64  
 2   Surname          165034 non-null  object 
 3   CreditScore      165034 non-null  int64  
 4   Geography        165034 non-null  object 
 5   Gender           165034 non-null  object 
 6   Age              165034 non-null  float64
 7   Tenure           165034 non-null  int64  
 8   Balance          165034 non-null  float64
 9   NumOfProducts    165034 non-null  int64  
 10  HasCrCard        165034 non-null  float64
 11  IsActiveMember   165034 non-null  float64
 12  EstimatedSalary  165034 non-null  float64
 13  Exited           165034 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 17.6+ MB


In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.drop(['id','CustomerId','Surname','Geography','Gender'],axis=1).corrwith(df['Exited']).abs().sort_values(ascending=False)

In [ ]:
churn_rate = df['Exited'].value_counts()/len(df)*100
plt.figure(figsize=(5,5))
plt.pie(churn_rate, labels=['Exited','Not Exited'])
plt.show()

In [ ]:
df1 = df.copy()
df1['Exited'] = df1['Exited'].astype(str)

fig, axes = plt.subplots(1,2, figsize=(15,5))

sns.countplot(x='Gender', hue='Exited', palette='viridis', data=df1, ax=axes[0])
axes[0].set_title("Gender of exited customers")

sns.countplot(x='Geography', hue='Exited', palette='viridis', data=df1, ax=axes[1])
axes[1].set_title("Geography of exiting customers")

plt.show()

In [ ]:
df1 = df.copy()
df1['Exited'] = df1['Exited'].astype(str)

fig, axes = plt.subplots(1,2, figsize=(15,5))

sns.countplot(x='NumOfProducts', hue='Exited', palette='viridis', data=df1, ax=axes[0])
axes[0].set_title("NumOfProducts")

sns.countplot(x='IsActiveMember', hue='Exited', palette='viridis', data=df1, ax=axes[1])
axes[1].set_title("IsActiveMember")

plt.show()

In [ ]:
df1 = df.copy()
df1['Exited'] = df1['Exited'].astype(str)

fig, axes = plt.subplots(1,2, figsize=(15,5))

sns.countplot(x='Tenure', hue='Exited', palette='viridis', data=df1, ax=axes[0])
axes[0].set_title("Tenure")

sns.countplot(x='HasCrCard', hue='Exited', palette='viridis', data=df1, ax=axes[1])
axes[1].set_title("HasCrCard")

plt.show()

In [ ]:
df_1 = df.drop(['id','CustomerId','Surname','Geography','Gender'], axis=1)
%matplotlib inline
df_1.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
%matplotlib inline
df_p=df[['EstimatedSalary']]
df_p['EstimatedSalary'].hist(bins=50, figsize=(9,6))
plt.grid(which='both')
plt.minorticks_on()
plt.show()

In [ ]:
%matplotlib inline
df_p=df[['Age']]
df_p['Age'].hist(bins=50, figsize=(9,6))
plt.grid(which='both')
plt.minorticks_on()
plt.show()

In [ ]:
%matplotlib inline
df_p=df[['CreditScore']]
df_p['CreditScore'].hist(bins=50, figsize=(9,6))
plt.grid(which='both')
plt.minorticks_on()
plt.show()

In [ ]:
%matplotlib inline
df_p=df[df['Balance']>1]
df_p['Balance'].hist(bins=50, figsize=(9,6))
plt.grid(which='both')
plt.minorticks_on()
plt.show()

# **ML**

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv',index_col='id')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv',index_col='id')

In [ ]:
df_train['Exited']

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# robust scaler

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("scaler", StandardScaler())
    ])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
        ("cat_encoder", OneHotEncoder()),
    ])

In [ ]:
from sklearn.compose import ColumnTransformer
target = ['Exited']
cat_attribs=[ 'Gender','Geography']
num_attribs=[ 'CreditScore', 'Age',
       'Tenure', 'Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [ ]:
X = df_train.drop('Exited',axis=1)
y = df_train['Exited']

In [ ]:
X_transformed = preprocess_pipeline.fit_transform(
    X[num_attribs + cat_attribs])

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_transformed,y,test_size=0.2,random_state=42)

# **Train the RandomForestClassifier model**

In [ ]:
# Train the RF model
RF_model = RandomForestClassifier(n_estimators=9)
RF_model.fit(X_train, y_train)

# Modelni baholaymiz
y_pred = RF_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Model aniqligi:", accuracy_score(y_test,y_pred))

## confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_mat, annot=True,fmt="g")
plt.show()

## ROC curve
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='ROC curve')
display.plot()
plt.show()

from sklearn.model_selection import cross_val_score
# Model evaluation using cross-validation
print(f"cross validation score {cross_val_score(RF_model, X_train, y_train, cv=5, scoring='accuracy').mean()}")

# **Train the XGBClassifier model**

In [ ]:
# Modelni yaratamiz (training)
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

# Modelni baholaymiz
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Model aniqligi:", accuracy_score(y_test,y_pred))

## confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_mat, annot=True,fmt="g")
plt.show()

## ROC curve
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='ROC curve')
display.plot()
plt.show()

from sklearn.model_selection import cross_val_score
# Model evaluation using cross-validation
print(f"cross validation score {cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='accuracy').mean()}")

# **Train the KNeighborsClassifier model**

In [ ]:
# Train the KNN model
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

# Modelni baholaymiz
y_pred = knn_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Model aniqligi:", accuracy_score(y_test,y_pred))

## confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_mat, annot=True,fmt="g")
plt.show()

## ROC curve
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='ROC curve')
display.plot()
plt.show()

from sklearn.model_selection import cross_val_score
# Model evaluation using cross-validation
print(f"cross validation score {cross_val_score(knn_model, X_train, y_train, cv=5, scoring='accuracy').mean()}")

# **Model evaluation & optimization**

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

In [ ]:
X_test_predict = preprocess_pipeline.fit_transform(
    df_test[num_attribs + cat_attribs])
X_test_predict

In [ ]:
Y_test_pred = xgb_model.predict(X_test_predict)
Y_test_pred

In [ ]:
y_result = df_test
y_result['Exited'] = Y_test_pred
y_result = y_result["Exited"]

In [ ]:
y_result

In [ ]:
y_result.to_csv('subm.csv')

# **Hyperparametrs**

In [ ]:
def hyperparameter_tune(base_model, parameters, n_iter, kfold, X=X_train, y=y_train):
    start_time = time.time()
    
    # Arrange data into folds with approx equal proportion of classes within each fold
    k = StratifiedKFold(n_splits=kfold, shuffle=False)
    
    optimal_model = RandomizedSearchCV(base_model,
                            param_distributions=parameters,
                            n_iter=n_iter,
                            cv=k,
                            n_jobs=-1,
                            random_state=42)
    
    optimal_model.fit(X, y)
    
    stop_time = time.time()

    scores = cross_val_score(optimal_model, X, y, cv=k, scoring="accuracy")
    
    print("Elapsed Time:", time.strftime("%H:%M:%S", time.gmtime(stop_time - start_time)))
    print("====================")
    print("Cross Val Mean: {:.3f}, Cross Val Stdev: {:.3f}".format(scores.mean(), scores.std()))
    print("Best Score: {:.3f}".format(optimal_model.best_score_))
    print("Best Parameters: {}".format(optimal_model.best_params_))
    
    return optimal_model.best_params_, optimal_model.best_score_

In [ ]:
base_model = RandomForestClassifier(n_jobs=-1,
                                   random_state=42)

lots_of_parameters = {
    "max_depth": [3, 5, 10, None],
    "n_estimators": [100, 200, 300, 400, 500],
    "max_features": randint(1, 3),
    "criterion": ["gini", "entropy"],
    "bootstrap": [True, False],
    "min_samples_leaf": randint(1, 4)
}

parameters = {
    "max_depth": [3, 5, 10, None],
    "n_estimators": [100, 200, 300, 400, 500]
}

best_params, best_score = hyperparameter_tune(base_model, parameters, 10, 5, X_train, y_train)

In [ ]:
scores = []
folds = range(2, 8)

for i in folds:
    print("\ncv = ", i)
    best_params, best_score = hyperparameter_tune(base_model, parameters, 10, i, X_train, y_train)
    scores.append(best_score)